In [1]:
import glob
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
from scipy.stats import variation, kurtosis, skew
from sklearn.preprocessing import normalize

In [2]:
distancesDir = 'decade_distances_complete/*.pickle'
distances_raw = []
for name in glob.glob(distancesDir):
    year = int(name[26:30])
    with open(name, 'rb') as f:
        distances_raw.append((year, pickle.load(f)))
        f.close()

distances = [(yr, df) for yr, df in distances_raw if yr >= 1880 and yr <= 1945]
distances.sort(key=lambda i: i[0])

In [3]:
distancesDir = 'decade_distances_complete/*.pickle'
distances_raw = []
for name in glob.glob(distancesDir):
    year = int(name[26:30])
    with open(name, 'rb') as f:
        distances_raw.append((year, pickle.load(f)))
        f.close()

distances = [(yr, df) for yr, df in distances_raw if yr >= 1880 and yr <= 1945]
distances.sort(key=lambda i: i[0])

In [6]:
dfDict = {'year': [],
         'skew_JSD': [],
         'kurtosis_JSD': [],
         'skew_APD': [],
         'kurtosis_APD': []}

for yr, df in distances:
    apd = df['apd'].to_numpy()
    jsd = df['jsd'].to_numpy()
    
    # add yr to pdDict
    dfDict['year'].append(yr)
    
    # calculate jsd info
    dfDict['skew_JSD'].append(np.round(skew(jsd, nan_policy='omit'), 3))
    dfDict['kurtosis_JSD'].append(np.round(kurtosis(jsd, nan_policy='omit'), 3))
    
    # calculate apd info
    dfDict['skew_APD'].append(np.round(skew(apd, nan_policy='omit'), 3))
    dfDict['kurtosis_APD'].append(np.round(kurtosis(apd, nan_policy='omit'), 3))
    
df = pd.DataFrame(dfDict)

In [7]:
print(df)

    year  skew_JSD  kurtosis_JSD  skew_APD  kurtosis_APD
0   1880     1.293         1.984     0.344         1.239
1   1885     1.339         2.456     0.243         0.728
2   1890     1.549         3.149     0.122         1.018
3   1895     1.691         3.858    -0.288         2.664
4   1900     1.751         4.011    -0.068         1.882
5   1905     1.749         4.092    -0.016         1.381
6   1910     1.568         3.472     0.023         1.609
7   1915     1.379         2.469     0.093         2.192
8   1920     1.559         3.090     0.016         1.070
9   1925     1.657         3.559     0.032         1.237
10  1930     1.330         2.419     0.006         1.820
11  1935     1.049         1.506    -0.012         1.421
12  1940     0.943         0.746     0.271         0.657
